# Market Analysis

In [1]:
%%capture
!pip install altair seaborn scipy scikit-learn plotly

In [2]:
import sys
import pandas as pd
import numpy as np
import copy
import ast
from collections import Counter

# Visualization packages
import altair as alt

# Append system path
sys.path = [p for p in sys.path if not p.endswith('../..')]  # Cleans duplicated '../..'
sys.path.insert(0, '../')  # This adds `src` to the path

from helpers import io
from analysis import analysis_util
from web_analysis import robots_util

%load_ext autoreload
%autoreload 2

In [9]:
# FPATH_TO_RELEVANT_URL_TOKENS = 'pretrain_data/relevant_url_token_counts.csv'
# FPATH_TO_ALL_URL_TOKENS = 'pretrain_data/all_url_token_counts.csv'
# FPATH_to_HEAD_ROBOTS = "robots_data/temporal_robots_head.json"
# FPATH_TO_RAND_ROBOTS = "robots_data/temporal_robots_rand_10k.json"
# FPATH_TO_TOS_DATA = "robots_data/tos_ai_scraping_policies.json"
# DIRPATHS_TO_ANNOTATED_TASKS = ["annotated_websites/Task 1", "annotated_websites/Task 2"]
# FPATH_SNAPSHOT_DATA = "robots_data/temporal_main_sites_current.json"
# START_DATES = "robots_data/domain_start_dates.json"

# COMPANIES_TO_ANALYZE = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta"]
# TEMPORAL_ANALYSIS_START_DATE = '2016-01-01'
# TEMPORAL_ANALYSIS_END_DATE = '2024-04-30'

EXCEL_FNAME_ROBOTS = "forecasted_robots_data.xlsx" # ???
EXCEL_FNAME_TOS = "forecasted_tos_data.xlsx" # ???
FPATH_TO_RELEVANT_URL_TOKENS = 'pretrain_data/relevant_url_token_counts.csv' # ???
FPATH_to_HEAD_ROBOTS = "robots_data/temporal_robots_head_test.json" # ???
FPATH_TO_RAND_ROBOTS = "data/robots/temporal_robots_rand_10k.json"
FPATH_TO_TOS_DATA = "data/GPT_analysis_results/tos_ai_scraping_policies.json"
FPATH_TO_TOS_LICENSE_DATA = "data/GPT_analysis_results/tos_license_policies.json"
FPATH_TO_TOS_COMPETE_DATA = "data/GPT_analysis_results/tos_competing_services_policies.json"
FPATH_TO_C4_TOKEN_ESTIMATES = "data/plotting/c4_total_token_estimates.csv" 
FPATH_TO_DOLMA_TOKEN_ESTIMATES = "data/plotting/dolma_total_token_estimates.csv" # ???
FPATH_TO_RF_TOKEN_ESTIMATES = "data/plotting/rf_total_token_estimates.csv" 
DIRPATHS_TO_ANNOTATED_TASKS = ["data/plotting/task_1", "data/plotting/task_2"]
START_DATES = "data/plotting/domain_start_dates.json"

ALL_COMPANIES_TO_TRACK = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta", "Internet Archive", "Google Search", "False Anthropic"]
COMPANIES_TO_ANALYZE = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta"]
TEMPORAL_ANALYSIS_START_DATE = '2016-01-01'
TEMPORAL_ANALYSIS_END_DATE = '2024-04-30'

In [4]:
agent_groups_to_track = robots_util.get_bot_groups(COMPANIES_TO_ANALYZE)
agents_to_track = robots_util.get_bots()

In [5]:
url_token_lookup = robots_util.URLTokenLookup(FPATH_TO_RELEVANT_URL_TOKENS) # 'c4', 'rf', 'dolma'
c4_url_to_counts = url_token_lookup.get_url_to_token_map("c4")
rf_url_to_counts = url_token_lookup.get_url_to_token_map("rf")
dolma_url_to_counts = url_token_lookup.get_url_to_token_map("dolma")
top_c4_urls = url_token_lookup.top_k_urls("c4", 2000)
top_rf_urls = url_token_lookup.top_k_urls("rf", 2000)
top_dolma_urls = url_token_lookup.top_k_urls("dolma", 2000)
random_10k_urls = url_token_lookup.get_10k_random_sample()
all_urls = set(random_10k_urls + top_c4_urls + top_rf_urls + top_dolma_urls)

# Load website snapshots for relevant URLs
# website_snapshots = robots_util.read_snapshots(FPATH_SNAPSHOT_DATA, all_urls)
website_start_dates = robots_util.read_start_dates(START_DATES, all_urls) # THIS WON'T WORK FOR THE 10k SAMPLE

Number of tokens in 2000 URLs: 18447797380 | 10.85% of c4
Number of tokens in 2000 URLs: 67098747294 | 15.56% of rf
Number of tokens in 2000 URLs: 429152555144 | 21.74% of dolma


In [6]:
# URL -> Date -> Robots.txt raw text
head_robots = io.read_json(FPATH_to_HEAD_ROBOTS)
random_10k_robots = io.read_json(FPATH_TO_RAND_ROBOTS)
joined_robots = copy.deepcopy(head_robots)
joined_robots.update(random_10k_robots)
robots_util.print_out_robots_info(head_robots)
robots_util.print_out_robots_info(random_10k_robots)

# {URL --> Date --> Agent --> Status}
url_robots_summary, agent_counter_df = robots_util.compute_url_date_agent_status(
    data=joined_robots, 
    # relevant_agents=agents_to_track)
    relevant_agents=[v for vs in agent_groups_to_track.values() for v in vs])

Num robot URLs loaded: 2985
Earliest time: 2016-01-01
Last time: 2024-04-19
Num robot URLs loaded: 6331
Earliest time: 2016-01-01
Last time: 2024-04-19


In [10]:
# URL --> Date --> ToS-suburl --> {"verdict": X, "evidence": Y}
# tos_policies = io.read_json(FPATH_TO_TOS_DATA)
# print(f"Num ToS URLs: {len(tos_policies)}")

# URL --> Date --> ToS-suburl --> {"verdict": X, "evidence": Y}
tos_policies = io.read_json(FPATH_TO_TOS_DATA)
tos_license_policies = io.read_json(FPATH_TO_TOS_LICENSE_DATA)
tos_compete_policies = io.read_json(FPATH_TO_TOS_COMPETE_DATA)
# tos_license_policies = robots_util.switch_dates_yearly_to_monthly(tos_license_policies)
print(f"Num ToS AI/Scraping URLs: {len(tos_policies)}")
print(f"Num ToS License URLs: {len(tos_license_policies)}")
print(f"Num ToS Compete URLs: {len(tos_compete_policies)}")

Num ToS AI/Scraping URLs: 3068
Num ToS License URLs: 3070
Num ToS Compete URLs: 3070


In [11]:
url_to_info = analysis_util.extract_url_annotations(DIRPATHS_TO_ANNOTATED_TASKS)
url_results_df = analysis_util.process_url_annotations(url_to_info)
url_results_df = analysis_util.encode_size_columns(url_results_df, url_token_lookup)
url_results_df = robots_util.encode_latest_tos_robots_into_df(
    url_results_df, tos_policies, tos_license_policies, tos_compete_policies,
    url_robots_summary, COMPANIES_TO_ANALYZE
)


6664 rows before filtering.
4029 rows after filtering. 1580 issues, 1055 unannotated.
<class 'collections.defaultdict'>
9312


In [12]:
c4_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_c4_urls, 
    "c4", 
    "pretrain_data/corpus_token_bucket_counts/c4_buckets.csv",
    url_token_lookup,
    verbose=False
)
rf_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_rf_urls, 
    "rf", 
    "pretrain_data/corpus_token_bucket_counts/rf_buckets.csv",
    url_token_lookup,
    verbose=False
)
dolma_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_dolma_urls, 
    "dolma", 
    "pretrain_data/corpus_token_bucket_counts/dolma_buckets.csv",
    url_token_lookup,
    verbose=False
)

Head sample size: 1634
Rand sample size: 1342
Head sample size: 1535
Rand sample size: 1342
Head sample size: 1324
Rand sample size: 1342


In [13]:
results_df = analysis_util.analyze_url_variable_correlations(
    url_results_df, [100, 500, 1000], 
    c4_estimates,
    rf_estimates,
    dolma_estimates,
)

# Convert the dataframe to a LaTeX table
latex_table = results_df.to_latex(index=True, escape=True, float_format="{:.1f}".format)
print(latex_table)

Num URLs in random sample: 1342
\begin{tabular}{lrrrrlll}
\toprule
 & Top 100 & Top 500 & Top 1000 & Random & C4 & RW & Dolma \\
\midrule
User Content & 19.8 & 16.8 & 17.5 & 15.1 & 38.3 & 54.5 & 25.5 \\
Paywall & 33.0 & 34.9 & 32.8 & 1.6 & 3.6 & 4.3 & 2.6 \\
Ads & 59.3 & 63.5 & 58.7 & 5.4 & 33.9 & 57.9 & 23.1 \\
Modality: Image & 98.9 & 98.8 & 99.1 & 95.0 & 103.2 & 126.7 & 114.8 \\
Modality: Video & 93.4 & 93.8 & 94.5 & 18.9 & 41.6 & 33.4 & 23.2 \\
Modality: Audio & 91.2 & 90.4 & 91.3 & 3.4 & 32.6 & 12.3 & 54.6 \\
Sensitive Content & 0.0 & 0.5 & 0.5 & 0.6 & 1.3 & 0.3 & 40.6 \\
services\_Academic & 8.8 & 8.9 & 10.9 & 3.8 & 2.9 & 2.9 & 1.9 \\
services\_Blogs & 3.3 & 4.3 & 3.7 & 15.1 & 23.1 & 24.3 & 18.8 \\
services\_E-Commerce & 8.8 & 10.6 & 10.3 & 10.6 & 31.5 & 46.1 & 16.4 \\
services\_Encyclopedia/Database & 16.5 & 13.0 & 11.3 & 0.5 & 11.4 & 4.0 & 51.2 \\
services\_Government & 2.2 & 2.9 & 2.6 & 1.1 & 0.8 & 1.0 & 0.6 \\
services\_News/Periodicals & 50.5 & 54.6 & 51.6 & 5.3 & 13.9 & 16.

In [14]:
def plot_altair_stacked_services_paywall(
    merged_df, 
    x_axis, 
    y_axis,
    stack_axis,
    stack_order,
    stack_colors,
    color="#1f77b4",
    legend_orient="top",
    axis_fontsize=10,
    labels_fontsize=10,
    font="arial",
    title="",
    title_x=None,
    title_y=None,
    labels_map=None,
    percentages=False,
    width=400, 
    height=400,
):
    """
    Plots a horizontal stacked bar chart with service categories on the y-axis
    and the specified x-value (instance rate or total tokens) on the x-axis.
    The bars are stacked based on the paywall and ads values.

    Args:
        df (pandas.DataFrame): The input dataframe with columns 'url', 'tokens',
            'services', 'paywall', and 'ads'.
        x_value (str, optional): The value to use for the x-axis. Can be 'instance_rate'
            or 'total_tokens'. Defaults to 'instance_rate'.
        width (int, optional): The width of the chart. Defaults to 800.
        height (int, optional): The height of the chart. Defaults to 400.

    Returns:
        altair.Chart: The stacked bar chart.
    """
    # Create a categorical data type with the desired ordering
    merged_df[stack_axis] = pd.Categorical(
        merged_df[stack_axis],
        categories=stack_order,
        ordered=True
    )
    merged_df = merged_df.sort_values(by=stack_axis)
    
    # Create the chart
    chart = alt.Chart(merged_df).mark_bar(color=color).encode(
        x=alt.X(
            f"{x_axis}:Q",
            title=x_axis.replace('_', ' ').title() if title_x is None else title_x,
            axis=alt.Axis(
                format=".0%" if percentages else "d",
                titleFontSize=axis_fontsize,
                titleFont=font,
                labelFontSize=labels_fontsize,
                labelFont=font,
            )
        ),
        y=alt.Y(
            f'{y_axis}:N', 
            sort='-x',
            # sort=services_ordered,
            title=title_y,
            axis=alt.Axis(
                titleFontSize=axis_fontsize,
                titleFont=font,
                labelFontSize=labels_fontsize,
                labelFont=font,
                labelExpr="datum.value" if (labels_map is None) else f"{labels_map}[datum.value]"
            )
        ),
        order="order:Q",
    ).properties(
        width=width,
        height=height,
        title=title
    )

    if stack_colors:
        chart = chart.encode(
            color=alt.Color(f"{stack_axis}:N",
                            title=None,
                            sort=stack_order,
                            scale=alt.Scale(
                                domain=stack_order,
                                range=stack_colors,
                            ),
                            legend=alt.Legend(
                                orient=legend_orient,
                                title=None,
                                labelFontSize=12,
                                labelFont=font,
                                labelPadding=5,
                                labelColor='black',
                                labelFontWeight='bold'
                            )
                           )
        )

    return chart

In [15]:
def prepare_altair_stacked_services_paywall(
    df,
    x_value='instance_count', 
    token_key='c4 tokens',
    paywall_ads_colors=['#8dd3c7', '#fb8072', '#ffffb3', '#bebada'],
    legend_orient="bottom-right",
    axis_fontsize=10,
    labels_fontsize=10,
    font="arial",
    title="",
    title_x="Total Tokens",
    title_y="Service Categories",
    labels_map=None,
    percentages=True,
    remove=[],
    width=800, 
    height=400,
):
    # Create a new column for the paywall/ads combination with custom labels
    df['paywall_ads'] = df.apply(lambda row: 'No Ads or Paywall' if not row['Paywall'] and not row['Ads'] else
                                             'Ads' if not row['Paywall'] and row['Ads'] else
                                             'Paywall' if row['Paywall'] and not row['Ads'] else
                                             'Both Paywall & Ads', axis=1)

    # Explode the 'services' column to create a row for each service
    df = df.explode('Services')

    # Calculate the instance rate and total tokens
    instance_counts = df.groupby(['Services', 'paywall_ads']).size().reset_index(name='instance_count')
    total_tokens = df.groupby(['Services', 'paywall_ads'])[token_key].sum().reset_index(name='total_tokens')

    # Merge the instance counts and total tokens
    merged_df = instance_counts.merge(total_tokens, on=['Services', 'paywall_ads'])

    merged_df["pct_tokens"] = merged_df["total_tokens"] / merged_df.total_tokens.sum()

    merged_df['percent'] = merged_df.groupby(['Services', 'paywall_ads'])[x_value].transform('sum') / merged_df.groupby('Services')[x_value].transform('sum') * 100
    services_ordered = merged_df.groupby('Services')[x_value].sum().sort_values(ascending=False).index.tolist()

    paywall_ads_order = ['No Ads or Paywall', 'Ads', 'Paywall', 'Both Paywall & Ads']

    # Remove given columns
    merged_df.drop(merged_df[merged_df["Services"].isin(remove)].index, inplace=True)
    return plot_altair_stacked_services_paywall(
        merged_df,
        x_axis="pct_tokens", # Otherwise x_value
        y_axis="Services",
        stack_axis="paywall_ads",
        stack_order=paywall_ads_order,
        stack_colors=paywall_ads_colors,
        legend_orient=legend_orient,
        axis_fontsize=axis_fontsize,
        labels_fontsize=labels_fontsize,
        font=font,
        title=title,
        title_x=title_x,
        title_y=title_y,
        labels_map=labels_map,
        percentages=percentages,
        width=width,
        height=height
    )

In [16]:
def prepare_altair_wildchat(
    fpath,
    color="#1f77b4",
    axis_fontsize=10,
    labels_fontsize=10,
    font="arial",
    title="",
    title_x="Count",
    title_y="Service Categories",
    labels_map=None,
    percentages=False,
    remove=[],
    width=400, 
    height=400,
):
    df = pd.read_csv(fpath)
    category_map = {
        'Coding Composition': [
            'Coding composition (fixing, debugging, or help)',
            'Coding composition',
        ],
        'Explanation and Reasoning': [
            'Asking for an explanation, reasoning, or help solving a puzzle, or math problem',
            'Asking for an explanation, reasoning, or help solving a puzzle or math problem',
        ],
        'Creative Composition': [
            'Creative composition',
            'Creative composition (such as role-playing, fictional story writing or continuation)'
        ],
        'Academic Composition': [
            'Academic composition (such as non-fiction essay writing, continuation, or fixing)',
            'Academic composition'
        ],
        'General Information': ['General informational requests'],
        'Sexual/Illegal Content': ['Sexual or illegal content requests'],
        'Translation': ['Translation'],
        'Brainstorming and Planning': ['Brainstorming, planning, or ideation'],
        'Self-help & self-harm': [
            'Self-help, advice seeking, or self-harm',
            'Self-help, advice seeking'
        ],
        'Organization Information': [
            'Information requests specifically about organizations, companies, or persons'
        ],
        'E-commerce Information': [
            'E-commerce or information requests about products and purchasing'
        ],
        'News': [
            'News or recent events informational requests',
        ],
        'Other': ['Other', np.nan],
    }
    RELEVANT_CATEGORIES = {
        "News": "Relevance to Websites",
        "E-commerce Information": "Relevance to Websites",
        "Organization Information": "Relevance to Websites",
        "Academic Composition": "Relevance to Websites",
        "Sexual/Illegal Content": "NSFW Content",
    }
    
    inverse_mapper = {v: k for k, vs in category_map.items() for v in vs}

    def safe_literal_eval(val):
        if "[" not in val:
            return [val]
        else:
            try:
                return ast.literal_eval(val)
            except (ValueError, SyntaxError):
                print(val)
                return None  # or some other default value or handling mechanism
    
    df['Types of Service'] = df['Types of Service'].apply(safe_literal_eval)

    # df['Types of Service'] = df['Types of Service'].apply(ast.literal_eval)
    df = df.explode('Types of Service')

    cat_df = []
    for category, count in Counter(df['Types of Service'].tolist()).most_common():
        if isinstance(category, float) and np.isnan(category):
            category = np.nan
        cat_df.append({
            "category": inverse_mapper[category],
            "count": count,
            "stack": RELEVANT_CATEGORIES.get(inverse_mapper[category], "Less Relevant to Websites"),
        })
    cat_df = pd.DataFrame(cat_df)

    cat_df["pct_count"] = cat_df["count"] / cat_df["count"].sum()
    
    # Remove given columns
    cat_df.drop(cat_df[cat_df["category"].isin(remove)].index, inplace=True)
    return plot_altair_stacked_services_paywall(
        cat_df,
        x_axis="pct_count", 
        y_axis="category", 
        stack_axis="stack", 
        stack_order=["Relevance to Websites", "Less Relevant to Websites", "NSFW Content"], 
        stack_colors=[],
        color=color,
        axis_fontsize=axis_fontsize,
        labels_fontsize=labels_fontsize,
        font=font,
        title=title,
        title_x=title_x,
        title_y=title_y,
        labels_map=labels_map,
        percentages=percentages,
        width=width,
        height=height
    )

In [17]:
# CHANGE HERE HOW THE PLOT LOOKS
services_paywall_chart = prepare_altair_stacked_services_paywall(
    url_results_df,
    x_value='total_tokens',
    paywall_ads_colors=['#ffffd1', '#add8b7', '#64b4c2', '#335da3'],
    legend_orient="bottom-right",
    axis_fontsize=18,
    labels_fontsize=15,
    font="times",
    title="(a) Web Domain Services",
    title_x="% of Total Tokens",
    title_y="Service Categories",
    labels_map={"News/Periodicals": "News", "Encyclopedia/Database": "Encyclopedia", "Organization/Personal Website": "Org Site", "Social Media/Forums": "Social Media", "Other": "Other", "Academic": "Academic", "E-Commerce": "E-Commerce", "Blogs": "Blogs", "Government": "Government"},
    percentages=True,
    remove=[],
    width=350, 
    height=150,
)
services_paywall_chart.show()

alt.Chart(...)

In [22]:
# CHANGE HERE HOW THE PLOT LOOKS
wildchat_chart = prepare_altair_wildchat(
    "wildchat_analysis_results.csv",
    color="#ac8298",
    axis_fontsize=18,
    labels_fontsize=15,
    font="times",
    title="(b) Real ChatGPT Uses",
    title_x="Estimated % of Queries",
    title_y=None, # Manuel: I'm removing it only in the second plot because it looks better when concatenated, but you can add it back
    labels_map={"Creative Composition": "Creative Composition", "Sexual/Illegal Content": "Sexual Content", "Brainstorming and Planning": "Brainstorming & Planning", "Explanation and Reasoning": "Explanation & Reasoning", "General Information": "General Information", "Coding Composition": "Coding Composition", "Academic Composition": "Academic Composition", "Translation": "Translation", "Organization Information": "Organization Info", "E-commerce Information": "E-commerce Info", "Self-help & self-harm": "Self-help/harm", "News": "News"},
    percentages=True,
    remove=["Other", "Self-help & self-harm"],
    width=350, 
    height=150,
)
wildchat_chart.show()

alt.Chart(...)

In [23]:
# CHANGE HERE THE ORDER OF THE PLOTS
alt.hconcat(services_paywall_chart, wildchat_chart).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0 # Remove the frame around the chart
).resolve_legend(
    color='independent'
)

alt.HConcatChart(...)

In [ ]:
# Calculate underlying percents. Look at random/head, dolma/rf/c4.

In [ ]:
# df_uc = url_results_df[url_results_df["sample"] != "random"]

In [ ]:
# # Normalize by token count
# # ax = plot_modality_stacked_bar(url_results_df, normalize_by='token_count')

# # Normalize by URL count
# ax = plot_modality_stacked_bar(df_uc, normalize_by='url_count')

In [24]:
# Explode the 'domains' column
dfx = url_results_df.explode('Domains')

# Get the unique domain categories
domain_categories = dfx['Domains'].unique()

# Create a new column for each domain category with True/False values
for category in domain_categories:
    url_results_df[f'domain_{category}'] = url_results_df['Domains'].apply(lambda x: category in x)

# Filter the DataFrame to only include rows with video modality
df_with_videos = url_results_df[url_results_df['Modality: Video']]

# Calculate the portion of URLs with videos for each domain category
portions_by_category = {}
for category in domain_categories:
    category_col = f'domain_{category}'
    total_videos = len(df_with_videos)
    videos_in_category = df_with_videos[category_col].sum()
    portion = videos_in_category / total_videos
    portions_by_category[category] = portion

# Print the results
tot_portions = sum(portions_by_category.values())
for category, portion in portions_by_category.items():
    print(f"{category}: {100 * portion / tot_portions :.2f}%")
# print(sum(tot_portions))

News: 30.00%
Entertainment & Culture: 21.00%
Legal & Policy: 4.23%
Science, Academia, & Technology: 10.62%
General Information & Education: 11.34%
Books: 1.79%
Business & E-Commerce: 11.17%
Social Media/Forums: 4.23%
Other: 2.78%
Blogs: 2.85%
